In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")

text = text[:100000]


Taille du vocabulaire : 6745


In [2]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(5, len(sequence_data)):
    words = sequence_data[i-5:i+1]
    sequences.append(words)

sequences = np.array(sequences)




In [3]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  
y = sequences[:, -1]  
y = to_categorical(y, num_classes=vocab_size)





In [4]:


model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(Bidirectional(GRU(120)))
model.add(Dense(vocab_size, activation='softmax'))

model.build(input_shape=(None, X.shape[1])) 

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=80, verbose=1)



Epoch 1/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.0310 - loss: 7.0738
Epoch 2/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0564 - loss: 5.9692
Epoch 3/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0800 - loss: 5.5707
Epoch 4/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1056 - loss: 5.1886
Epoch 5/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.1266 - loss: 4.8142
Epoch 6/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1517 - loss: 4.4634
Epoch 7/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.1775 - loss: 4.0800
Epoch 8/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.2216 - loss: 3.7026
Epoch 9/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2839 - loss: 3.3204
Epoch 10/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3480 - loss: 2.9291
Epoch 11/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.4166 - loss: 2.5980
Epoch 12/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 1

In [6]:
input_text = " It is a truth universally"
predict_next_words = 45


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-5:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

 It is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife however little known the feelings or views of such a man may be on his first entering a neighbourhood this truth is so well fixed in the
